<a href="https://colab.research.google.com/github/dianranii/capstone-project/blob/main/Capstone_Projek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import zipfile
from PIL import Image
from io import BytesIO
import ipywidgets as widgets
from IPython.display import display
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
zip_file = r"/content/drive/My Drive/DATASET.zip"

In [5]:
path_folder = "path_fold"

In [6]:
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(path_folder)

In [7]:
folder_path="path_fold"
files = os.listdir(folder_path)
print(files)

['TEST', 'TRAIN']


In [8]:
train_dir=os.path.join(folder_path,'TRAIN')
test_dir=os.path.join(folder_path,'TEST')

In [9]:
train_dir_O = os.path.join(train_dir, 'O')
train_dir_E = os.path.join(train_dir, 'E')
train_dir_R = os.path.join(train_dir, 'R')

In [10]:
test_dir_O = os.path.join(test_dir, 'O')
test_dir_E = os.path.join(test_dir, 'E')
test_dir_R = os.path.join(test_dir, 'R')

In [11]:
def convert_images_to_jpeg(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)

            if file.endswith(('.png', '.bmp', '.gif', '.tiff', '.jpg')):
                try:
                    with Image.open(file_path) as img:
                        new_file_path = os.path.splitext(file_path)[0] + ".jpeg"
                        img.convert('RGB').save(new_file_path, "JPEG")
                        os.remove(file_path)
                except Exception as e:
                    print(f"Error converting {file_path}: {e}")

In [12]:
train_dir_convert=convert_images_to_jpeg(train_dir)
test_dir_convert=convert_images_to_jpeg(test_dir)

In [13]:
"""def convert_to_jpeg(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                with Image.open(file_path) as img:
                    new_file_path = os.path.splitext(file_path)[0] + ".jpeg"
                    img.convert('RGB').save(new_file_path, "JPEG")
            except Exception as e:
                print(f"Error converting {file_path}: {e}")"""

'def convert_to_jpeg(directory):\n    for root, dirs, files in os.walk(directory):\n        for file in files:\n            file_path = os.path.join(root, file)\n            try:\n                with Image.open(file_path) as img:\n                    new_file_path = os.path.splitext(file_path)[0] + ".jpeg"\n                    img.convert(\'RGB\').save(new_file_path, "JPEG")\n            except Exception as e:\n                print(f"Error converting {file_path}: {e}")'

In [14]:
"""def load_and_preprocess_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [150, 150])
    image = image / 255.0
    return image"""

'def load_and_preprocess_image(image_path):\n    image = tf.io.read_file(image_path)\n    image = tf.image.decode_jpeg(image, channels=3)\n    image = tf.image.resize(image, [150, 150])\n    image = image / 255.0\n    return image'

In [15]:
"""def load_images_from_directory(directory_path):
    images = []
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)
        if file_path.endswith(".jpg") or file_path.endswith(".png"):  # Pastikan hanya membaca file gambar
            image = load_and_preprocess_image(file_path)
            images.append(image)
    return images"""

'def load_images_from_directory(directory_path):\n    images = []\n    for filename in os.listdir(directory_path):\n        file_path = os.path.join(directory_path, filename)\n        if file_path.endswith(".jpg") or file_path.endswith(".png"):  # Pastikan hanya membaca file gambar\n            image = load_and_preprocess_image(file_path)\n            images.append(image)\n    return images'

In [16]:
"""train_dir=load_images_from_directory(train_dir)
test_dir=load_images_from_directory(test_dir)"""

'train_dir=load_images_from_directory(train_dir)\ntest_dir=load_images_from_directory(test_dir)'

In [17]:
def create_model():
    '''Creates a CNN with 4 convolutional layers'''
    model = tf.keras.models.Sequential([
        tf.keras.Input(shape=(150, 150, 3)),
        tf.keras.layers.Rescaling(1./255),
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    return model

In [18]:
model = create_model()
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)                │ (None, 150, 150, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 148, 148, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 74, 74, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 72, 72, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 36, 36, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 34, 34, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 17, 17, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 15, 15, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 7, 7, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 6272)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │       3,211,776 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             513 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,453,121 (13.17 MB)

 Trainable params: 3,453,121 (13.17 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
# Define fill mode.
FILL_MODE = 'nearest'

# Create the augmentation model.
data_augmentation = tf.keras.Sequential([
    # Specify the input shape.
    tf.keras.Input(shape=(150,150,3)),
    # Add the augmentation layers
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2, fill_mode=FILL_MODE),
    tf.keras.layers.RandomTranslation(0.2,0.2, fill_mode=FILL_MODE),
    tf.keras.layers.RandomZoom(0.2, fill_mode=FILL_MODE)
    ])

In [20]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(150, 150),
    batch_size=20,
    label_mode='binary'
    )

Found 22564 files belonging to 2 classes.


In [21]:
validation_dataset = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=(150, 150),
    batch_size=20,
    label_mode='binary'
    )

Found 2513 files belonging to 2 classes.


In [22]:
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics = ['accuracy']
    )

In [23]:
SHUFFLE_BUFFER_SIZE = 1000
PREFETCH_BUFFER_SIZE = tf.data.AUTOTUNE

train_dataset_final = train_dataset.cache().shuffle(SHUFFLE_BUFFER_SIZE).prefetch(PREFETCH_BUFFER_SIZE)
validation_dataset_final = validation_dataset.cache().prefetch(PREFETCH_BUFFER_SIZE)

In [24]:
history = model.fit(
    train_dataset_final,
    epochs=15,
    validation_data=validation_dataset_final,
    verbose=2
    )

Epoch 1/15
1129/1129 - 46s - 41ms/step - accuracy: 0.8029 - loss: 0.4617 - val_accuracy: 0.8707 - val_loss: 0.3510
Epoch 2/15
1129/1129 - 15s - 13ms/step - accuracy: 0.8345 - loss: 0.3908 - val_accuracy: 0.9093 - val_loss: 0.2494
Epoch 3/15
1129/1129 - 15s - 13ms/step - accuracy: 0.8549 - loss: 0.3586 - val_accuracy: 0.8727 - val_loss: 0.3121
Epoch 4/15
1129/1129 - 21s - 18ms/step - accuracy: 0.8647 - loss: 0.3377 - val_accuracy: 0.8770 - val_loss: 0.3239
Epoch 5/15
1129/1129 - 21s - 18ms/step - accuracy: 0.8761 - loss: 0.3185 - val_accuracy: 0.9180 - val_loss: 0.2492
Epoch 6/15
1129/1129 - 20s - 18ms/step - accuracy: 0.8811 - loss: 0.3051 - val_accuracy: 0.8735 - val_loss: 0.3452
Epoch 7/15
1129/1129 - 15s - 14ms/step - accuracy: 0.8877 - loss: 0.2877 - val_accuracy: 0.8846 - val_loss: 0.3298
Epoch 8/15
1129/1129 - 20s - 18ms/step - accuracy: 0.8966 - loss: 0.2750 - val_accuracy: 0.8659 - val_loss: 0.4159
Epoch 9/15
1129/1129 - 15s - 14ms/step - accuracy: 0.9092 - loss: 0.2493 - val_a

In [25]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [26]:
uploader = widgets.FileUpload(accept="image/*", multiple=True)
display(uploader)
out = widgets.Output()
display(out)

def file_predict(filename, file, out):

    try:
        image = Image.open(file)
        if image.format not in ['JPEG']:
            image = image.convert('RGB')
            filename = filename.split('.')[0] + ".jpeg"
            image.save(filename, 'JPEG')
    except Exception as e:
        with out:
            print(f"Error memproses file {filename}: {e}")
        return

    image = image.resize((150, 150))
    image = np.array(image)
    image = np.expand_dims(image, axis=0)

    prediction = model.predict(image, verbose=0)[0][0]

    with out:
        if prediction > 0.5:
            print(f"{filename} adalah sampah anorganik")
        else:
            print(f"{filename} adalah sampah organik")

def on_upload_change(change):

    items = uploader.value
    with out:
        print("Item yang diunggah:", list(items.keys()))
    for filename, file_info in items.items():
        try:
            content = file_info['content']
            file_jpgdata = BytesIO(content)
            file_predict(filename, file_jpgdata, out)
        except Exception as e:
            with out:
                print(f"Kesalahan dengan file {filename}, kesalahan: {e}")



uploader.observe(on_upload_change, names='value')

FileUpload(value={}, accept='image/*', description='Upload', multiple=True)

Output()

In [27]:
model.save("klasifikasi_sampah.h5")